## PuLPとGurobi/Pythonによる混合整数最適化問題のモデリングのテクニック

久保幹雄　(東京海洋大学）

* 混合整数最適化ソルバー（モデラー）Gurobi, PuLP
* 混合整数最適化ラッパー mypulp
* 簡単な例題
* 論理制約の定式化のテクニック
* 具体例
* 定式化のコツ
* ベンチマーク比較

### (混合）整数最適化ソルバー(モデラー）
-----------
* ### Gurobi（商用，アカデミックフリー）のソルバー
  * 独自のPythonインターフェイス（「あたらしい数理最適化」（近代科学社）で採用）
 * 凸2次（制約）整数，2次錐最適化
* ### PuLP （MITライセンス）のモデラー
  * メインソルバーはCBC(COIN Branch & Cut; EPLライセンス），その他様々なソルバーと連携(MPSフォーマット経由）

### 共通Model $\Rightarrow$ Wrapper $\Rightarrow$ Modeler  $\Rightarrow$ Solver

 <div align="center">
<img src="wrapper.jpg" alt="Drawing" style="width: 700px;"/>




基本クラスと主要メソッド

 <div align="center">
<img src="solver.jpg" alt="Drawing" style="width: 500px;"/>


## 例題 パズル（整数最適化）

鶴と亀と蛸が何匹かずついる
頭の数を足すと $32$，足の数を足すと $80$ になる．
亀と蛸の数の和を一番小さくするような匹数を求めよ．

\begin{array}{l c c }
  \mbox{minimize}    &  y + z          \\
  \mbox{subject to}  & x + y + z  = 32 \\
                     & 2x +4y +8z = 80 \\
               & x,y,z  \mbox{ は非負の整数} 
\end{array}



In [4]:
#from gurobipy import *
from mypulp import *
model = Model("puzzle")
x = model.addVar(vtype="I", name="x")
y = model.addVar(vtype="I", name="y")
z = model.addVar(vtype="I", name="z")
model.update()

model.addConstr(x + y + z == 32, "Heads")
model.addConstr(2*x + 4*y + 8*z == 80, "Legs")

model.setObjective(y + z, GRB.MINIMIZE)

model.optimize()

print("Opt. Val.=",model.ObjVal)
print("(x,y,z)=",(x.X,y.X,z.X))

Opt. Val.= 4.0
(x,y,z)= (28.0, 2.0, 2.0)


In [6]:
#鶴亀蛸キメラ算
#from gurobipy import *
from mypulp import *
model = Model("puzzle")
x = model.addVar(vtype="I", name="x")
y = model.addVar(vtype="I", name="y")
z = model.addVar(vtype="I", name="z")
zz = model.addVar(vtype="I", name="zz")
model.update()

model.addConstr(x + y + z + 2*zz == 32, "Heads")
model.addConstr(2*x + 4*y + 8*z + 3*zz == 99, "Legs")

model.setObjective(y + z, GRB.MINIMIZE)

model.optimize()

print("Opt. Val.=",model.ObjVal)
print("(x,y,z,zz)=",(x.X,y.X,z.X,zz.X))

Opt. Val.= 6.0
(x,y,z,zz)= (24.0, 0.0, 6.0, 1.0)


## 例題 多制約ナップサック問題（0-1整数最適化）

* $n$ 個のアイテム，$m$本の制約
* 各々のアイテム $j=1,2,\ldots,n$ の価値 $v_j ~(\geq 0)$，
* アイテム $j$ の制約 $i=1,2,\ldots,m$ に対する重み $a_{ij}~(\geq 0)$
* 制約 $i$ に対する制約の上限値 $b_i~(\geq 0)$ 

<div align="center">
<img src="mkp1.jpg" alt="Drawing" style="width: 600px;"/>



* アイテムの番号の集合 $I =\{1,2,\ldots,n \}$
* 制約の番号の集合 $J =\{1,2,\ldots,m\}$ 
* アイテム $j$ をナップサックに詰めるとき $1$，
それ以外のとき $0$ になる $0$-$1$ 変数 $x_j$

定式化

\begin{array}{l l l}
   \mbox{maximize}     & \displaystyle\sum_{j \in J} v_j x_j          &  \\
   \mbox{subject to}  &  \displaystyle\sum_{j \in J} a_{ij} x_j \leq b_i & \forall i \in I    \\
                       & x_j \in \{0,1\} &     \forall j \in J
\end{array}

In [2]:
#from gurobipy import *
from mypulp import *

J,v = multidict({1:16, 2:19, 3:23, 4:28})
a = {(1,1):2,    (1,2):3,    (1,3):4,    (1,4):5,
     (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200,
    }
I,b = multidict({1:7, 2:10000})
    
model = Model('mkp')
x = {}
for j in J:
    x[j] = model.addVar(vtype='B', name='x({0})'.format(j))
model.update()
     
for i in I:
    model.addConstr(quicksum(a[i,j]*x[j] for j in J) <= b[i], 'Capacity({0})'.format(i))

model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)

model.optimize()
print('Optimal value=',model.ObjVal)

EPS = 1.e-6
for v in model.getVars():
    if v.X > EPS:
        print( v.VarName,v.X)

Optimal value= 42.0
x(2) 1.0
x(3) 1.0


## 論理条件の例
-----------

上の多制約 $0$-$1$ ナップサック問題の例題に対する付加条件：

*  極小クマ $x_1$ と大クマ $x_4$ は仲が悪いので，同時に持って逃げてはいけない．

$$x_1 +x_4 \leq 1$$ 

*  極小クマ $x_1$ は小クマ $x_2$ が好きなので，極小クマを持って逃げるときには必ず小クマももって逃げなければならない．

$$x_1 \leq x_2$$  

*  極小クマ $x_1$，小クマ $x_2$，中クマ $x_3$ のうち，少なくとも2つは持って逃げなければならない．

$$ x_1 +x_2+x_3 \geq 2$$ 

# 離接制約
--------

** either or の条件** : 何れか1つが満たされていなければならない

* $2x_1 + x_2  \leq 30, x_1 + 2x_2  \leq 30$ の2 本の制約の何れか1つが成立する

* 大きな数 $M$ と $0$-$1$変数 $y$ を用いて，

$$2x_1 + x_2  \leq 30 +My, \ \ \  x_1 + 2x_2  \leq 30+M(1-y)$$



* $2x_1 + x_2  \leq 30, \ \ \  x_1 + 2x_2  \leq 30, \ \ \ 5x_1+x_2 \leq 50$
の3 本の制約の何れかが成立するという条件

  * 3 つの$0$-$1$変数 $y_1, y_2, y_3$ を用いて，

$$2x_1 + x_2  \leq 30 +M(1-y_1)$$
$$x_1 + 2x_2  \leq 30+M(1-y_2)$$
$$5x_1+x_2 \leq 50+M(1-y_3)$$

$$y_1 +y_2 + y_3 \geq 1$$


## if A then B 条件
-------------

「制約Aが成立している場合には，制約Bも成立しなければならない」という条件


「if A then B」は「NOT A or B」と同値


|   A  |  B |   if A then B |  NOT A |  NOT A or B |
|:----:|:----:|:----:|:---:|:---:
|   偽 |  偽  |     真    |     真  |       真 |
|   偽 |  真  |     真    |     真  |       真 |
|   真 |  偽  |     偽    |     偽  |       偽 |
|   真 |  真  |     真    |     偽  |       真 |


* if A then B の条件： 

$$\mbox{if  $x_1 +x_2  \leq 10$  then  $x_1 \leq 5$ }$$

* NOT A or B を離接制約で表現：

$$x_1 + x_2 > 10 - My, \ \ \ x_1 \leq 5 + M(1-y)$$ 


問題点：数理最適化ソルバーでは，より大きい（$>$）と以上（$\geq$）の制約を区別しない

* $x_1 + x_2 > 10$ は $x_1 + x_2 \geq 10$ と同値

微少な値 $\epsilon >0$ を用いて，

$$x_1 + x_2 \geq 10 +\epsilon - My$$ 

* $x_1, x_2$ が整数の値なら $\epsilon=1$: 

$$x_1 + x_2 \geq 11- My$$


## 半連続変数
--------------


 $0$ もしくはある範囲内の連続変数

* 変数 $x$ が半連続変数で $[LB, UB]$ の範囲

* $\Rightarrow$ $0$-$1$ 変数 $y$ を用いて 

 $$LB y \leq x \leq UB y$$ 


## 最小値の最小化
------------------

* 最大値の最小化は線形最適化で解説；最小値の最小化
  * 新しい実数変数 $z$ と $0$-$1$ 変数 $y$ を導入

$$3x_1 + 4x_2  \geq z, \ \   2x_1 + 7x_2  \geq z$$

$$3x_1 + 4x_2  \leq z- My, \ \   2x_1 + 7x_2  \leq z - M(1-y)$$


* 線形関数 $\displaystyle\sum_{j} a_{ij} x_j  (i=1,2,\ldots,m)$ の最小値を最小化
  * $0$-$1$ 変数 $y_i (i=1,2,\ldots,m)$ を導入

$$\displaystyle\sum_{j} a_{ij} x_j \geq z \ \ \ i=1,2,\ldots,m$$

$$\displaystyle\sum_{j} a_{ij} x_j \leq z - M(1-y_i)\ \ \ i=1,2,\ldots,m$$

$$\displaystyle\sum_{i} y_i =1$$

* 例：線形最適化問題の最初の例
* 最も儲かる売値の設定は？

|丼 | トンコケ  |  コケトン  | ミックス   | 
|:--:|:----:|:----:|:---:|
|1   | 1500 | 1800| 3000 |
|2   | 1800 | 2200| 2900 |
|3   | 1300 | 1700| 3300 |

In [3]:
from mypulp import *
model = Model('max-max')
M=999999
x, y ={}, {}
for i in range(1,4):
    x[i]=model.addVar(vtype='I', name='x{0}'.format(i))
    y[i]=model.addVar(vtype='B', name='y{0}'.format(i))
z  = model.addVar(name='z')
model.update()      
model.addConstr(2*x[1] +   x[2] + x[3] <= 100)
model.addConstr(  x[1] + 2*x[2] + x[3] <= 100)
model.addConstr(                  x[3] <= 50)
model.addConstr(15*x[1] + 18*x[2] + 30*x[3] <= z )
model.addConstr(18*x[1] + 22*x[2] + 29*x[3] <= z )
model.addConstr(13*x[1] + 17*x[2] + 33*x[3] <= z )
model.addConstr(15*x[1] + 18*x[2] + 30*x[3] >= z -M*(1-y[1]))
model.addConstr(18*x[1] + 22*x[2] + 29*x[3] >= z -M*(1-y[2]))
model.addConstr(13*x[1] + 17*x[2] + 33*x[3] >= z -M*(1-y[3]))
model.addConstr( y[1] + y[2] + y[3] == 1 )
model.setObjective(z, GRB.MAXIMIZE)
model.optimize()
if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 2147.0
x1 16.0
x2 17.0
x3 50.0
y1 0.0
y2 0.0
y3 1.0
z 2147.0


## 絶対値の最大化

* 実数変数 $x$ の絶対値 $|x|$ を最大化（最小化は線形最適化で解説）
    * 2 つの新しい非負の実数変数 $y$ と $z$ を導入
    * $x=y-z$ を追加
    * $|x|$ をすべて $y+z$ で置き換え
    * $y$ と $z$ が同時に正にならない（大きな数 $M$ と $0$-$1$ 変数 $\xi$ を使う）
  
  $$y \leq M \xi$$
  
  $$z \leq M (1-\xi)$$
    

In [2]:
# Ansewr of Practice Problem  (werewolf 1)
# 正直族，嘘つき族と狼男 1
from mypulp import *

m=Model()
x, w = {}, {} 
for i in range(3):
    x[i]=m.addVar(name="x({0})".format(i), vtype="B" )
    w[i]=m.addVar(name="w({0})".format(i), vtype="B" )
m.addConstr(x[0] == w[0] ) #A: I'm a werewolf. 
m.addConstr(x[1] == w[1] ) #B: Me, too. 
#c: At most one person is honest. 
m.addConstr(quicksum( w[i] for i in range(3)) <= 1+ 3*(1-x[2]) ) #True => at most one.
m.addConstr(quicksum( w[i] for i in range(3)) >= 2- 3*x[2] )     #False => at least two.
m.addConstr(quicksum( w[i] for i in range(3)) == 1 ) #One is a werewolf.
m.optimize()
for i in range(3):
    print(x[i].X, w[i].X)

0.0 0.0
0.0 0.0
1.0 1.0


In [3]:
# Ansewr of Practice Problem  (werewolf 2)
# 正直族，嘘つき族と狼男 2
m=Model()
x, w = {}, {} 
for i in range(3):
    x[i]=m.addVar(name="x({0})".format(i), vtype="B" )
    w[i]=m.addVar(name="w({0})".format(i), vtype="B" )
#A: At least one is a liar, i.e., at most two person are honest.  
m.addConstr(quicksum( x[i] for i in range(3)) <= 2+ 1*(1-x[0]) )
#If A tells a lie, all are honest. 
m.addConstr(quicksum( x[i] for i in range(3)) >= 3 - 3*x[0] )
#B: C is honest. 
m.addConstr(x[1]==x[2] )
#At least one is a werewolf and he is honest. 
m.addConstr(quicksum( w[i] for i in range(3)) >= 1)
for i in range(3):
    m.addConstr(w[i] <= x[i]) 
m.optimize()
for i in range(3):
    print(x[i].X, w[i].X)

1.0 1.0
0.0 0.0
0.0 0.0


In [ ]:
# 座席割り当て問題
from mypulp import *
# = 0 means Male, =1 means female
Family={ "Isono":[0,1,0,1], "Bakabon": [0,1,0,0], "Nohara":[0,1,0,1],
        "Nobi":[0,1,0], "Hoshi":[0,1,0], "Rei": [0,1,0], "Zabi":[0,1,0,1,0,0,0]}

n=7 #number of tables
Table =list(range(n))

m=Model()

x={}    #assignment variable of the member of the families to the tables
deviation ={} #deviation of the number of females from the average 
male, female ={}, {}
total_female=0
total_male=0
for f in Family:
    for i,j in enumerate(Family[f]):
        for k in Table:
            x[f,i,k]=m.addVar(name="x({0},{1},{2})".format(f,i,k), vtype="B")
        male[f,i]= 1-j
        female[f,i] =j
        total_male+=1-j
        total_female+=j
       # print(x[f,i,0], "=", female[f,i])
print("num of females=",total_female)
print("num of males=",total_male)
total=total_female+total_male

for k in range(n):
    deviation[k]=m.addVar(name="dev({0})".format(k) )
    
m.update()

for f in Family:
    for i,j in enumerate(Family[f]):
        m.addConstr( quicksum( x[f,i,k] for k in range(n)) == 1 )

for f in Family:
    for i1,j1 in enumerate(Family[f]):
        for i2,j2 in enumerate(Family[f]):
            if i1 < i2:
                for k in range(n):
                    m.addConstr(x[f,i1,k]+x[f,i2,k]<=1)
average= total_female / n
print("average number of females per table=",average)

for k in range(n):
   m.addConstr(deviation[k]>=average- quicksum(female[f,i]*x[f,i,k] for f in Family for i in range(len(Family[f]))) ) 
   m.addConstr(deviation[k]>=quicksum(female[f,i]*x[f,i,k] for f in Family for i in range(len(Family[f]))) -average)    
   
m.setObjective( quicksum( deviation[k] for k in Table),GRB.MINIMIZE)
m.optimize()

for f in Family:
    for i,j in enumerate(Family[f]):
        for k in Table:
            if x[f,i,k].X >0.01:
                print(f,i,k)
for k in Table:
    print(deviation[k].X)

## 具体例
------------

* 起動停止問題（生産スケジューリング，スタッフスケジューリング）
* その他の代表的な最適化問題（時間があれば．．．）

## 起動停止問題

* 各日の電力需要を満たすように．．．
* 時間ごとの発電量の決定
* 一度火を入れると $\alpha$ 時間は停止できない
* 一度停止すると $\beta$ 時間は再稼働できない
* 稼働費用は非線形（2次）関数
* 稼働中の発電量の上下限 $U_i, L_i$ 


### 集合

* $P$: 発電所（ユニット）の集合

### 変数

* $y_t^p$: 期 $t$ に発電所 $p$ が稼働するとき $1$ 
    
### 新しい変数を導入

* $z_t^p$: 期 $t$ に発電所 $p$ が稼働を開始(switch-on)するとき $1$

* $w_t^p$:  期 $t$ に発電所 $p$ が停止を開始(switch-off)するとき $1$

* 基本構造：発電所の起動停止問題 

= スタッフスケジューリング問題 

= 生産計画の小バケットロットサイズ決定問題

\begin{array}{l l } 
  %z_t^p \leq y_t^p               &  \forall p \in P, t=1,2, \ldots,T \\
  z_t^p -w_{t}^p = y_t^p -y_{t-1}^p &  \forall p \in P, t=1,2, \ldots,T 
\end{array}
    
|  変数  | 　t-3　 | t-2  |  t-1 |　t　 | t+1 | t+2 | t+3 |
|:----:  |:------:|:------:|:-----:|:-----:|:---:|:---:|:---:|:---:|
|   $y_t$ |  0  |     0    |    0  |     1 |  1    |  1 |  0  | 
|  $z_t$  | 0  |     0    |    0  |     1 |  0    |  0 |  0  |  
|  $w_t$ |  0  |     0    |    0  |     0 |  0    |  0 |  1  |  

### 開始したら最低でも $\alpha$ 期 は連続稼働：

$\Rightarrow$ 「期 $t$ に稼働していない」　 **ならば**　 「$t-\alpha+1$ から $t$ までは開始できない」

$\Rightarrow$ $y_t^p$ ならば $z_{s}^p=0$ ($\forall s=t-\alpha+1,\ldots,t$)  

$$
\displaystyle\sum_{s=t-\alpha+1}^t z_{s}^p \leq y_t^p  \ \ \  \forall t=\alpha,\alpha+1, \ldots,T
$$

#### 弱い定式化：

$\Rightarrow$ 「期 $t$ に開始した」 **ならば** 「$t$ から$t+\alpha+1$ までは稼働」

$$
\alpha z_{t}^p \leq \displaystyle\sum_{s=t}^{t+\alpha-1} y_t^p \ \ \  \forall t=1,2, \ldots,T+1-\alpha
$$



### 稼働を終了したら，再び稼働を開始するためには，$\beta$ 期以上：

$\Rightarrow$ 「期 $t$ に稼働した」　 **ならば** 「$t-\beta+1$ から $t$ までは終了できない」

$\Rightarrow$ $y_t^p=1$ ならば $w_{s}^p=0$ ($\forall s=t-\beta+1,\ldots,t$)

$$
\displaystyle\sum_{s=t-\beta+1}^{t} w_{s}^p \leq 1-y_t^p  \ \ \  \forall t=\beta,\beta+1, \ldots,T
$$

#### 弱い定式化：

$\Rightarrow$ 「期 $t$ に停止した」　**ならば**  「$t$から $t+\beta-1$ までは稼働しない」

$$
\beta w_{t}^p \leq \displaystyle\sum_{s=t}^{t+\beta-1} (1-y_t^p) \ \ \  \forall t=1,2, \ldots,T+1-\beta
$$



## 生産スケジューリング


### 集合

* $P$: 品目の集合

### 変数

* $y_t^p$: 期 $t$ に品目 $p$ が段取りを行うとき $1$ 
    
### 新しい変数を導入

* $z_t^p$: 期 $t$ に品目 $p$ に対する生産を開始するとき $1$

* $w_t^p$:  期 $t-1$ に品目 $p$ に対する生産を終了するとき $1$


### 各期における段取りが高々1回のときの最も強い定式化：

\begin{array}{l l } 
  \displaystyle\sum_{p \in P} y_{t}^p \leq 1  & \forall t=1,2, \ldots,T \\
  z_t^p \leq y_t^p               &  \forall p \in P, t=1,2, \ldots,T \\
  z_t^p -w_{t-1}^p = y_t^p -y_{t-1}^p &  \forall p \in P, t=1,2, \ldots,T \\
  y_{t-1}^p +z_{t}^p + \displaystyle\sum_{q: q \neq p} (y_t^q-z_t^q) \leq 1 &  \forall p \in P, t=1,2,\ldots,T 
\end{array}

* 最後の式は理論的な側面定義不等式（なくても良いし，実験的にはない方が良い）
 

### 実験結果（スタッフスケジューリング）

* 総スタッフ数は91 名
* 平日約35 名（ピーク時間帯勤務約20 名）程度
* 土日祝日約40 名（ピーク時間帯勤務約25 名）程度
* 1 日の期数は33 期（7 時～23 時30 分までを30 分刻み）
* 業務は5 種類＋休憩
* スタッフ数の過不足に対するペナルティ費用
* 休憩に関する詳細な設定あり

$\Rightarrow$ 3600秒で最適解

「スケジューリング問題に関する諸研究」，田頭弘光（海洋大学修士論文）


### 実験結果（起動停止問題）

* 100ユニットのベンチマーク問題（Kazarlis benchmarks）
* 2次費用は（逐次）区分的線形近似
* 15000秒で最適解
* 付加条件にも対応（非凸型の費用関数：バルブ効果）

"Unit commitment with valve-point loading effect,"
Joao Pedro Pedroso, Mikio Kubo, and  Ana Viana,
INESC Porto and Instituto Superior de Engenharia, 
Technical Report Series: DCC-2014-05

### ベンチマーク比較

Benchmarks for Optimization Software by Hans Mittelmann

http://plato.la.asu.edu/bench.html

ベンチマーク問題 MIPLIB2010 http://miplib.zib.de/miplib2010.php

* CBC-2.8.10: CBC
* CPLEX-12.6.1: CPLEX
* SCIP/cplex, spx-3.1.1: Parallel development version of SCIP (FSCIP+CPLEX/SOPLEX)
* GUROBI-6.0.0 GUROBI
* XPRESS-7.8.0: XPRESS


### 比較

|    4スレッド | CBC |   CPLEX |   FSCIPC |  FSCIPS |  GUROBI |  XPRESS|
|--:|----:|----:|----:|----:|----:|----:|
| 幾何平均   | 19.4 |   1.05  |   8.19  |   12.7   |    1  |     1.30 |  
| 87問中解けた問題 |   58  |    86  |     72  |     67  |     86  |     86 |
  


## Questions, Comments, Suggestions?